In [ ]:
# Predicting whether integer is nearby (within 16)

In [1]:
# imports
from scipy.sparse import csr_matrix
import numpy as np
import pickle

from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation
from keras.callbacks import EarlyStopping, ModelCheckpoint

Using TensorFlow backend.


In [2]:
# DNN
loss_function = 'binary_crossentropy' # 'categorical_crossentropy' or 'binary_crossentropy'
final_activation = 'softmax'          # 'softmax' or 'sigmoid'
inner_activation = 'relu'
'''Use categorical and softmax if targets are exclusive
   Use binary and sigmoid if inputs can have multiple target labels'''

dropout_prob = 0.1    # dropout on hidden layer
dropout_input = 0.01   # dropout on input features
'''Note that dropout often improves performance but also increases training time'''

num_nodes = 500       # number of nodes in the hidden layer

In [3]:
# NN CODE
'''Constructs a simple shallow neural net with dropout.
   The training code automatically uses 5% of the training data as a validation set,
   and stops training when the loss on the validation set starts to increase, which
   reduces the risk of overfitting.
   We automatically save the model config to model.json and the model weights to weights.hdf5
'''
#construct model
clf = Sequential()
clf.add(Dropout(dropout_input, input_shape=(16,)))
clf.add(Dense(num_nodes, input_dim = 16, activation= inner_activation))
clf.add(Dropout(dropout_prob))
clf.add(Dense(2, activation= final_activation))

#compile and train model
clf.compile(optimizer='Adam', loss= loss_function)

In [4]:
#stop if validation get worse
earlyStopping=EarlyStopping(monitor='val_loss', patience=10)

In [5]:
import random

In [6]:
a=[random.randint(0,256) for x in range(5000)]
b=[random.randint(0,256) for x in range(5000)]
c=[[0,1] if abs(xa-xb)<17 else [1,0] for xa,xb in zip(a,b)]

In [7]:
np.unpackbits(np.array([[1],[2],[3]],dtype=np.uint8), axis=1)

array([[0, 0, 0, 0, 0, 0, 0, 1],
       [0, 0, 0, 0, 0, 0, 1, 0],
       [0, 0, 0, 0, 0, 0, 1, 1]], dtype=uint8)

In [8]:
a_vec = np.unpackbits(np.array([[xa] for xa in a],dtype=np.uint8),axis=1)
b_vec = np.unpackbits(np.array([[xb] for xb in b],dtype=np.uint8),axis=1)


In [10]:
"""
Stack left and right to vector
>>> np.hstack([
    np.array([[0,0,0,1],[1,0,1,0]]), 
    np.array([[0,0,0,1],[0,1,1,0]])
])

array([[0, 0, 0, 1, 0, 0, 0, 1],
       [1, 0, 1, 0, 0, 1, 1, 0]])

"""
ab_vec = np.hstack([
    a_vec,b_vec
])
c_vec = c

train = ab_vec[0:4000]
tag_t = c_vec[0:4000]
validate = ab_vec[4000:]
tag_v = c_vec[4000:]

In [11]:
clf.fit(train, tag_t, epochs=100, 
#         class_weight={0:1,1:2}, 
        validation_data=(validate,tag_v), callbacks=[earlyStopping],
        verbose=1)

Train on 4000 samples, validate on 1000 samples
Epoch 1/100
4000/4000 [==============================] - 2s - loss: 0.3888 - val_loss: 0.3504
Epoch 2/100
4000/4000 [==============================] - 0s - loss: 0.3440 - val_loss: 0.2968
Epoch 3/100
4000/4000 [==============================] - 0s - loss: 0.2752 - val_loss: 0.2166
Epoch 4/100
4000/4000 [==============================] - 0s - loss: 0.2148 - val_loss: 0.1716
Epoch 5/100
4000/4000 [==============================] - 0s - loss: 0.1789 - val_loss: 0.1443
Epoch 6/100
4000/4000 [==============================] - 0s - loss: 0.1686 - val_loss: 0.1289
Epoch 7/100
4000/4000 [==============================] - 0s - loss: 0.1577 - val_loss: 0.1154
Epoch 8/100
4000/4000 [==============================] - 0s - loss: 0.1446 - val_loss: 0.1127
Epoch 9/100
4000/4000 [==============================] - 0s - loss: 0.1423 - val_loss: 0.1045
Epoch 10/100
4000/4000 [==============================] - 0s - loss: 0.1308 - val_loss: 0.0976
Epoch 11/10

In [13]:
# Accuracy
np.mean([x==y for x,y in
(zip(np.argmax(clf.predict(validate), axis=1), 
              np.argmax(tag_v, axis=1)))
])

0.97199999999999998

In [14]:
# Accuracy for [0,1] to validate imbalance class
np.mean([x==y for x,y in
(zip(np.argmax(clf.predict(validate), axis=1), 
              np.argmax(tag_v, axis=1)))
if y==1])

0.9568965517241379

In [23]:
from sklearn.metrics import f1_score
y_true = np.argmax(tag_v, axis=1)
y_pred = np.argmax(clf.predict(validate), axis=1)

print(f1_score(y_true, y_pred, average=None))


[ 0.984  0.888]
